Veamos un ejemplo que Gerón muestra en su libro. Supongamos que queremos saber si el dinero hace felices a las personas.

Para esto, vamos a descargar los datos del "Índice para una Vida Mejor" del sitio web de la OCDE y las estadísticas del Banco Mundial sobre el producto interior bruto (PIB) per cápita. Luego unimos las tablas y las ordenamos por PIB per cápita.

*   **OCDE**: Organización internacional que reúne países para mejorar sus políticas económicas y sociales.
Banco Mundial: Institución que financia proyectos y apoya el desarrollo
*   **Banco Mundial**: Institución que financia proyectos y apoya el desarrollo económico de países.
*   **PIB / PIB per cápita**: El PIB mide todo lo que produce un país; el PIB per cápita muestra cuánto produce o gana, en promedio, cada persona.

In [7]:
import urllib.request # urllib es un paquete de python que sirve para "consultar en la web", y request es un módulo específico dentro de esta librería
from pathlib import Path # pathlib es un módulo estándar de Python que sirve para trabajar con rutas de archivos y directorios de forma más intuitiva que con cadenas de texto

# Crea un objeto Path que representa la ruta ./datasets/lifesat.
datapath = Path() / "datasets" / "lifesat"

# Crea la carpeta datasets/lifesat.
# parents=True → crea todas las carpetas intermedias si no existen.
# exist_ok=True → no da error si la carpeta ya existe.
datapath.mkdir(parents=True, exist_ok=True)

# Define la URL base donde están los archivos que queremos descargar.
# En este caso, el fichero está en el repositorio de Gerón
data_root = "https://github.com/ageron/data/raw/main/"

# Itera sobre los nombres de los archivos que queremos descargar.
# Vamos a descargar los dos archivos explicados antes
for filename in ("oecd_bli.csv", "gdp_per_capita.csv"):

    # Comprueba si el archivo ya existe en nuestra carpeta local. Si existe, no lo descarga otra vez.
    if not (datapath / filename).is_file():

        # Muestra un mensaje indicando qué archivo se está descargando.
        print("Downloading", filename)

        # Construye la URL completa del archivo a descargar.
        url = data_root + "lifesat/" + filename

        # Descarga el archivo desde la URL y lo guarda en la ruta local especificada.
        urllib.request.urlretrieve(url, datapath / filename)

# Para que este código funcione en Google Colab:
# Las rutas de archivos no se crean automáticamente, pero Path funciona igual.
# Los archivos se guardan en la carpeta temporal de Colab (/content/ por defecto).
# No necesitas instalar urllib ni pathlib, son parte de Python estándar.

Ya tenemos los ficheros descargados, ahora podemos leerlos:

In [10]:
# Importamos la librería pandas, que sirve para trabajar con datos en forma de tablas
import pandas as pd

# Leemos los archivos CSV que descargamos antes (están en la carpeta 'datasets/lifesat')
# Cada archivo se convierte en una "tabla" (DataFrame) dentro de Python
oecd_bli = pd.read_csv(datapath / "oecd_bli.csv")
gdp_per_capita = pd.read_csv(datapath / "gdp_per_capita.csv")

# Elegimos el año con el que queremos trabajar
gdp_year = 2020

# Guardamos los nombres de las columnas que queremos usar más adelante
gdppc_col = "GDP per capita (USD)"   # Producto Interno Bruto por persona (en dólares)
lifesat_col = "Life satisfaction"    # Nivel de satisfacción con la vida

# Filtramos la tabla para quedarnos solo con los datos del año 2020
gdp_per_capita = gdp_per_capita[gdp_per_capita["Year"] == gdp_year]

# Eliminamos las columnas que no nos interesan ("code" y "year")
# axis=1 indica que estamos borrando columnas (no filas)
gdp_per_capita = gdp_per_capita.drop(["Code", "Year"], axis=1)

# Cambiamos los nombres de las columnas para que sean más claros
gdp_per_capita.columns = ["Country", gdppc_col]

# Hacemos que la columna "Country" (país) sea el índice de la tabla
# Así cada fila estará identificada por el nombre del país
gdp_per_capita.set_index("Country", inplace=True)

# Mostramos las primeras filas de la tabla para ver cómo quedó
gdp_per_capita.head()



,GDP per capita (USD)
Country,
Afghanistan,1978.961579
Africa Eastern and Southern,3387.594670
Africa Western and Central,4003.158913
Albania,13295.410885
Algeria,10681.679297


Ahora vamos a preprocesar los datos del Índice para una Vida Mejor (OECD BLI) para quedarnos solo con la columna “Satisfacción con la vida”.

In [11]:
# Filtramos las filas del DataFrame para quedarnos solo con los datos
# donde la columna "INEQUALITY" tiene el valor "TOT"
# ("TOT" significa "Total", es decir, sin distinguir por género, edad u otros factores)
oecd_bli = oecd_bli[oecd_bli["INEQUALITY"] == "TOT"]

# Reorganizamos la tabla con pivot() para que:
# - Cada país quede en una fila (index="Country")
# - Cada tipo de indicador (por ejemplo, "Life satisfaction", "Income", etc.) sea una columna
# - Los valores numéricos de esos indicadores estén en las celdas (values="Value")
oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")

# Mostramos las primeras filas del nuevo DataFrame para comprobar el resultado
oecd_bli.head()


Indicator,Air pollution,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Feeling safe walking alone at night,Homicide rate,Household net adjusted disposable income,Household net wealth,Housing expenditure,...,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Stakeholder engagement for developing regulations,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education
Country,,,,,,,,,,,,,,,,,,,,,
Australia,5.0,NaN,81.0,13.04,73.0,63.5,1.1,32759.0,427064.0,20.0,...,49126.0,95.0,NaN,85.0,2.7,502.0,14.35,91.0,93.0,21.0
Austria,16.0,0.9,85.0,6.66,72.0,80.6,0.5,33541.0,308325.0,21.0,...,50349.0,92.0,1.6,70.0,1.3,492.0,14.55,80.0,92.0,17.0
Belgium,15.0,1.9,77.0,4.75,63.0,70.1,1.0,30364.0,386006.0,21.0,...,49675.0,91.0,2.2,74.0,2.0,503.0,15.70,89.0,84.0,19.3
Brazil,10.0,6.7,49.0,7.13,61.0,35.6,26.7,NaN,NaN,NaN,...,NaN,90.0,NaN,NaN,2.2,395.0,NaN,79.0,73.0,16.2
Canada,7.0,0.2,91.0,3.69,73.0,82.2,1.3,30854.0,423849.0,22.0,...,47622.0,93.0,2.6,88.0,2.9,523.0,14.56,68.0,91.0,17.3


Ahora combinemos los datos de satisfacción con la vida y de PIB per cápita, quedándonos solo con las columnas PIB per cápita y Satisfacción con la vida.

In [12]:
# Unimos (fusionamos) las dos tablas: oecd_bli y gdp_per_capita
# La unión se hace usando los índices (en este caso, el nombre del país)
# Esto crea una nueva tabla con las columnas de ambas fuentes de datos
full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                              left_index=True, right_index=True)

# Ordenamos las filas del DataFrame según la columna del PIB per cápita
# Esto facilita analizar la relación entre el PIB y la satisfacción con la vida
full_country_stats.sort_values(by=gdppc_col, inplace=True)

# Nos quedamos únicamente con las columnas de interés:
# "PIB per cápita" y "Satisfacción con la vida"
full_country_stats = full_country_stats[[gdppc_col, lifesat_col]]

# Mostramos las primeras filas del DataFrame resultante
full_country_stats.head()


,GDP per capita (USD),Life satisfaction
Country,,
South Africa,11466.189672,4.7
Colombia,13441.492952,6.3
Brazil,14063.982505,6.4
Mexico,17887.750736,6.5
Chile,23324.524751,6.5


Para ilustrar el riesgo de sobreajuste (overfitting), Gerón utiliza solo una parte de los datos (todos los países con un PIB per cápita entre min_gdp y max_gdp). Más adelante, muestra los países que se habían omitido y demuestra que no siguen en absoluto la misma tendencia lineal.

Gerón emplea este ejemplo para mostrar cómo un modelo puede parecer muy preciso si se analiza solo con una parte limitada de los datos (por ejemplo, países con PIB medio). Sin embargo, cuando se incorporan los países que quedaron fuera (con PIB muy alto o muy bajo), la relación deja de ser lineal, evidenciando el problema del sobreajuste, que ocurre cuando un modelo se adapta demasiado a los datos de entrenamiento y no generaliza bien a nuevos casos.

In [13]:
# Definimos los límites inferior y superior del PIB per cápita
# Solo queremos analizar los países cuyo PIB per cápita esté dentro de este rango
min_gdp = 23_500
max_gdp = 62_500

# Filtramos la tabla para quedarnos únicamente con los países
# que tienen un PIB per cápita entre min_gdp y max_gdp (inclusive)
# La condición usa el operador "&" (AND) para combinar ambos filtros
country_stats = full_country_stats[
    (full_country_stats[gdppc_col] >= min_gdp) &
    (full_country_stats[gdppc_col] <= max_gdp)
]

# Mostramos las primeras filas del nuevo DataFrame con los países seleccionados
country_stats.head()


,GDP per capita (USD),Life satisfaction
Country,,
Russia,26456.387938,5.8
Greece,27287.083401,5.4
Turkey,28384.987785,5.5
Latvia,29932.493910,5.9
Hungary,31007.768407,5.6


En este punto sería recomendable guardar los resultados ya que nos ha costado algo de trabajo llegar a esta tabla.

In [14]:
# Guardamos el DataFrame 'country_stats' en un archivo CSV llamado 'lifesat.csv'
# Este archivo contiene solo los países dentro del rango de PIB per cápita definido (min_gdp a max_gdp)
country_stats.to_csv(datapath / "lifesat.csv")

# Guardamos el DataFrame completo 'full_country_stats' en otro archivo CSV llamado 'lifesat_full.csv'
# Este archivo incluye todos los países, sin aplicar ningún filtro
full_country_stats.to_csv(datapath / "lifesat_full.csv")
